In [1]:
import pandas as pd
import joblib
import os
import sys
import numpy as np
from pathlib import Path
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import precision_recall_curve
from pathlib import Path

src_path = os.path.abspath("../src")
sys.path.insert(0, src_path)

from pre_processamento import preprocessar_dataset, vetorizar_textos

# carregar dataframes (todos os csv em ~/data)
print("Carregando dados...")

caminhos_csv = list(Path("../data").rglob("*.csv"))

def carregar_csv_padronizado(caminho):
    df = pd.read_csv(caminho)
    if 'body' in df.columns:
        df['conteudo'] = df['body']
    elif 'conteudo' in df.columns:
        pass
    else:
        raise ValueError(f"Arquivo {caminho} não contém colunas válidas para conteúdo.")

    if 'label' not in df.columns:
        raise ValueError(f"Arquivo {caminho} não contém a coluna 'label'.")

    return df[['conteudo', 'label']]

dfs = []
for caminho in caminhos_csv:
    try:
        print(f"Tentando carregar: {caminho}")
        df = carregar_csv_padronizado(caminho)
        dfs.append(df)
        print(f"Carregado: {caminho}")
    except Exception as e:
        print(f"Erro ao carregar {caminho}: {e}")

if dfs:
    df_total = pd.concat(dfs, ignore_index=True)
    print(f"Dataset final combinado com {len(df_total)} entradas.")
else:
    df_total = pd.DataFrame()
    print("Nenhum CSV válido foi carregado.")

# pre processa conteúdo textual
print("Aplicando limpeza de texto...")
df_total = preprocessar_dataset(df_total)

# vetorização com TF-IDF
print("Vetorizando textos com caractere n-grams...")
X, vectorizer = vetorizar_textos(df_total['conteudo'], analyzer='char_wb', ngram_range=(3, 5), max_features=5000)
y = df_total['label']

# balanceamento de classes
print("Aplicando SMOTE para balanceamento de classes...")
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# separação de treino e teste
print("Separando treino e teste...")
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# treinamento #
print("Treinando modelo híbrido com VotingClassifier...")
model_lr = LogisticRegression(class_weight='balanced', max_iter=1000)
model_rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
ensemble_model = VotingClassifier(estimators=[('lr', model_lr), ('rf', model_rf)], voting='soft')
ensemble_model.fit(X_train, y_train)

print("Calculando limiar ótimo com base em F1-score...")
probas = ensemble_model.predict_proba(X_test)
spam_scores = [p[1] for p in probas]
labels_bin = [1 if lbl == 'spam' else 0 for lbl in y_test]
precision, recall, thresholds = precision_recall_curve(labels_bin, spam_scores)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Limiar ótimo calculado: {optimal_threshold:.2f}")


# salvar objetos #
os.makedirs('../models', exist_ok=True)
print("Salvando objetos e modelo final...")
joblib.dump(X, '../models/X_tfidf.pkl')
joblib.dump(y, '../models/y_labels.pkl')
joblib.dump(ensemble_model, '../models/modelo_final.pkl')
print("Modelo salvo com sucesso!")

joblib.dump(optimal_threshold, '../models/threshold.pkl')
joblib.dump(vectorizer, '../models/tfidf_vectorizer.pkl')
print("Finalizado com sucesso!")


[nltk_data] Downloading package stopwords to /home/diego/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Carregando dados...
Tentando carregar: ../data/novos_emails_pt.csv
Carregado: ../data/novos_emails_pt.csv
Tentando carregar: ../data/emails_preparados.csv
Carregado: ../data/emails_preparados.csv
Tentando carregar: ../data/emails_adicionais.csv
Carregado: ../data/emails_adicionais.csv
Dataset final combinado com 3372 entradas.
Aplicando limpeza de texto...
Vetorizando textos com caractere n-grams...
Aplicando SMOTE para balanceamento de classes...
Separando treino e teste...
Treinando modelo híbrido com VotingClassifier...
Calculando limiar ótimo com base em F1-score...
Limiar ótimo calculado: 0.71
Salvando objetos e modelo final...
Modelo salvo com sucesso!
Finalizado com sucesso!
